# Model tunning

In [1]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
import os 
os.chdir('drive/My Drive/Home_Credit_Default_Risk')

## Import libraries

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import gc

In [0]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
import pickle

## Load feature engineered data

In [0]:
data = pd.read_csv('home_features.csv')

In [0]:
train = data[data['TARGET'].notnull()]
test = data[data['TARGET'].isnull()]

del data
gc.collect()
print('Train shape: ',train.shape)
print('Test shape: ',test.shape)

Train shape:  (307507, 759)
Test shape:  (48744, 759)


## Drop columns with null value geater than 75%

In [0]:
missing = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
missing.head()

refused_RATE_INTEREST_PRIVILEGED_min     100.0
refused_RATE_INTEREST_PRIVILEGED_mean    100.0
refused_RATE_INTEREST_PRIVILEGED_max     100.0
refused_RATE_INTEREST_PRIMARY_mean       100.0
refused_RATE_INTEREST_PRIMARY_max        100.0
dtype: float64

In [0]:
train_miss = missing.index[missing>75]

In [0]:
train = train.drop(columns=train_miss)
test = test.drop(columns=train_miss)

print('Train shape: ',train.shape)
print('Test shape: ',test.shape)

Train shape:  (307507, 713)
Test shape:  (48744, 713)


In [0]:
train.drop(columns=['SK_ID_CURR','index','Unnamed: 0'],inplace=True)
test.drop(columns=['SK_ID_CURR','index','Unnamed: 0','TARGET'],inplace=True)

In [0]:
y = train['TARGET']
x = train.drop(columns=['TARGET'])

## Train test split, impute missing value and scale data

In [0]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2,random_state =10)

In [0]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

x_train = imputer.fit_transform(x_train)
x_test = imputer.transform(x_test)

scaler = MinMaxScaler(feature_range = (0, 1))
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
x_train_scale = pd.DataFrame(data=x_train_scale,columns=x.columns.tolist())
x_test_scale = pd.DataFrame(data=x_test_scale,columns=x.columns.tolist())
with open('train_test.pkl','wb') as f:
  pickle.dump((x_train_scale,x_test_scale,y_train,y_test),f)

In [0]:
"""with open('train_test.pkl','wb') as f:
  pickle.dump((x_train_scale,x_test_scale,y_train,y_test),f)"""
x_train_scale,x_test_scale,y_train,y_test = pickle.load(open('train_test.pkl','rb'))

In [0]:
x_test_scale.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,...,cc_AMT_DRAWINGS_CURRENT_min,cc_AMT_DRAWINGS_CURRENT_max,cc_AMT_DRAWINGS_CURRENT_mean,cc_AMT_DRAWINGS_CURRENT_sum,cc_AMT_DRAWINGS_OTHER_CURRENT_sum,cc_AMT_DRAWINGS_POS_CURRENT_sum,cc_AMT_INST_MIN_REGULARITY_min,cc_AMT_INST_MIN_REGULARITY_max,cc_AMT_INST_MIN_REGULARITY_mean,cc_AMT_INST_MIN_REGULARITY_sum,cc_AMT_PAYMENT_CURRENT_sum,cc_AMT_PAYMENT_TOTAL_CURRENT_min,cc_AMT_PAYMENT_TOTAL_CURRENT_max,cc_AMT_PAYMENT_TOTAL_CURRENT_mean,cc_AMT_PAYMENT_TOTAL_CURRENT_sum,cc_AMT_RECEIVABLE_PRINCIPAL_min,cc_AMT_RECEIVABLE_PRINCIPAL_max,cc_AMT_RECEIVABLE_PRINCIPAL_mean,cc_AMT_RECEIVABLE_PRINCIPAL_sum,cc_AMT_RECIVABLE_min,cc_AMT_RECIVABLE_max,cc_AMT_RECIVABLE_mean,cc_AMT_RECIVABLE_sum,cc_AMT_TOTAL_RECEIVABLE_min,cc_AMT_TOTAL_RECEIVABLE_max,cc_AMT_TOTAL_RECEIVABLE_mean,cc_AMT_TOTAL_RECEIVABLE_sum,cc_CNT_DRAWINGS_ATM_CURRENT_sum,cc_CNT_DRAWINGS_CURRENT_sum,cc_CNT_DRAWINGS_OTHER_CURRENT_sum,cc_CNT_DRAWINGS_POS_CURRENT_sum,cc_CNT_INSTALMENT_MATURE_CUM_sum,cc_SK_DPD_min,cc_SK_DPD_max,cc_SK_DPD_mean,cc_SK_DPD_sum,cc_SK_DPD_DEF_min,cc_SK_DPD_DEF_max,cc_SK_DPD_DEF_mean,cc_SK_DPD_DEF_sum
0,0.055277,0.029146,0.023569,0.000589,0.0,0.0,0.0,0.003831,0.0,0.173913,0.0454,0.0458,0.0462,0.0762,0.0757,0.0745,0.105263,0.157895,1.0,0.0210,0.0208,0.0190,0.675817,0.967400,0.924969,0.809250,0.744407,0.0,0.0,0.0,0.0,0.0,0.5,0.0690,0.0690,0.0690,0.746834,0.643153,0.538052,1.0,...,0.005491,0.032679,0.003059,0.007896,0.0,0.000000,0.0,0.024106,0.039431,0.019187,0.007485,0.0,0.005617,0.003621,0.006381,0.248675,0.066429,0.032785,0.028786,0.244012,0.071300,0.032992,0.029474,0.244012,0.071325,0.033003,0.029290,0.009464,0.003916,0.0,0.00000,0.006032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.107270,0.048652,0.046016,0.000473,0.0,0.0,0.0,0.000000,0.0,0.043478,0.0876,0.0864,0.0840,0.0762,0.0757,0.0745,0.000000,0.052632,1.0,0.0210,0.0208,0.0190,0.636246,0.890003,0.304710,0.696382,0.834306,0.0,0.0,0.0,0.0,0.0,0.0,0.1379,0.1379,0.1379,0.556488,0.787688,0.598522,1.0,...,0.005491,0.032679,0.003059,0.007896,0.0,0.000000,0.0,0.024106,0.039431,0.019187,0.007485,0.0,0.005617,0.003621,0.006381,0.248675,0.066429,0.032785,0.028786,0.244012,0.071300,0.032992,0.029474,0.244012,0.071325,0.033003,0.029290,0.009464,0.003916,0.0,0.00000,0.006032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.155396,0.177326,0.158249,0.002012,0.0,0.0,0.0,0.000000,0.0,0.086957,0.1278,0.1291,0.1303,0.0000,0.0000,0.0000,0.000000,0.052632,1.0,0.0144,0.0144,0.0145,0.824408,0.900832,0.668473,0.823388,0.945606,0.0,0.0,0.0,0.0,0.0,0.5,0.0690,0.0690,0.0690,0.193464,0.829443,0.212859,1.0,...,0.005491,0.032679,0.003059,0.007896,0.0,0.000000,0.0,0.024106,0.039431,0.019187,0.007485,0.0,0.005617,0.003621,0.006381,0.248675,0.066429,0.032785,0.028786,0.244012,0.071300,0.032992,0.029474,0.244012,0.071325,0.033003,0.029290,0.009464,0.003916,0.0,0.00000,0.006032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.216172,0.459888,0.382716,0.001127,0.0,0.0,0.0,0.000000,0.0,0.043478,0.0876,0.0864,0.0840,0.0778,0.0778,0.0807,0.052632,0.105263,0.0,0.0079,0.0080,0.0080,0.388557,0.730024,0.788523,0.467290,0.723573,0.0,0.0,0.0,0.0,0.0,0.5,0.1379,0.1379,0.1379,0.518870,0.834197,0.713545,1.0,...,0.005491,0.127660,0.014496,0.064460,0.0,0.026589,0.0,0.089239,0.190671,0.287481,0.083089,0.0,0.063109,0.019569,0.083658,0.248675,0.257200,0.185518,0.404244,0.244012,0.267266,0.186834,0.397206

## Model tunning and selection.
Here we will try some models as below
* Logistic regression
* SGD Classifier 
* Random forest
* LGBM Classifier

## Logistic Regression

In [0]:
param = {'penalty':['l1','l2'],
        'C':[0.1,0.001,0.0001,0.00001]}
        

In [0]:
log_reg = LogisticRegression()
clf = GridSearchCV(log_reg,param_grid=param,n_jobs=-1,verbose=1)
clf.fit(x_train_scale,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  7.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.1, 0.001, 0.0001, 1e-05],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [0]:
clf.best_params_

{'C': 0.1, 'penalty': 'l2'}

In [0]:
log_reg = LogisticRegression(C=0.1,penalty='l2')
log_reg.fit(x_train_scale,y_train)
y_pred = log_reg.predict_proba(x_test_scale)[:,1]
score = roc_auc_score(y_test,y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
print(score)

0.7640211457290654


## SGD Classifier

In [0]:
param = {
    'loss':['hinge','log'],
    'penalty':['l1','l2','elasticnet'],
    'alpha':[0.1,0.001,0.0001],
    'fit_intercept':[True,False],
    'learning_rate':['optimal','adaptive']
}

In [0]:
sgd = SGDClassifier(eta0=0.1)
clf = GridSearchCV(sgd,param_grid=param,cv=5,n_jobs=-1,verbose=1)
clf.fit(x_train_scale,y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 64.2min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 134.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.1, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5,
                                     random_state=None, shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'alpha': [0.1, 0.001, 0.0001],
                         'fit_intercept': [True, False],
                         'learning_rate': ['optimal', 'adaptive'],
                     

In [0]:
print(clf.best_score_)
clf.best_params_

0.9193309079083758


{'alpha': 0.0001,
 'fit_intercept': False,
 'learning_rate': 'adaptive',
 'loss': 'log',
 'penalty': 'l2'}

#### Fit model on best parameters

In [0]:
scores =[]
sgd = SGDClassifier(loss='log',penalty='l2',learning_rate='adaptive',fit_intercept=False,alpha=0.0001,eta0=0.1,verbose=0)
for i in range(5):
  x_tr,x_te, y_tr,y_te = train_test_split(x_train_scale,y_train,test_size=0.1,random_state=i*10)
  print('Fold '+str(i))
  sgd.fit(x_tr,y_tr)
  y_pred = sgd.predict_proba(x_te)[:,1]
  score = roc_auc_score(y_te,y_pred)
  print(str(i) +') score = ',score)
  scores.append(score)

Fold 0
0) score =  0.7774451537193686
Fold 1
1) score =  0.7650878288691845
Fold 2
2) score =  0.7683897800149332
Fold 3
3) score =  0.7624985291445495
Fold 4
4) score =  0.7652513751052232


In [0]:
sgd.fit(x_train_scale,y_train)

y_pred =sgd.predict_proba(x_test_scale)[:,1]
score = roc_auc_score(y_test,y_pred)

print('Auroc score: ',score)

Auroc score:  0.7624007447132439


## Random Forest

Here we will use random forest to elemenate some features which are less important and see if removal of those features improves score or not. Take less data points for faster execution.

In [0]:
x_tr,x_te,y_tr,y_te = train_test_split(x_train_scale,y_train,test_size=0.9)
x_tr.shape

(24600, 709)

In [0]:
rnd = RandomForestClassifier(n_estimators=100,random_state=10,n_jobs=-1)
rnd.fit(x_train_scale,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [0]:
features = x.columns.tolist()
importance = rnd.feature_importances_

feat_imp = pd.DataFrame({'features':features,'importance':importance}).sort_values(by='importance')

In [0]:
feat_imp.head()

,features,importance
143,NAME_INCOME_TYPE_Student,8.007677e-07
629,pos_NAME_CONTRACT_STATUS_Canceled_mean,3.670227e-06
289,buro_CREDIT_ACTIVE_Bad debt_mean,4.843239e-06
199,ORGANIZATION_TYPE_Industry: type 8,8.317519e-06
443,prev_NAME_CASH_LOAN_PURPOSE_Refusal to name th...,9.635125e-06


In [0]:
feat_imp.tail()

,features,importance
23,DAYS_EMPLOYED,0.006404
22,DAYS_BIRTH,0.006560
36,EXT_SOURCE_1,0.007417
38,EXT_SOURCE_3,0.013292
37,EXT_SOURCE_2,0.017414


#### Remove feature with zero importance.

In [0]:
feat_imp = feat_imp[feat_imp['importance']!=0]

In [0]:
new_feat = feat_imp['features'].tolist()
import pickle
with open('imp_feat.pkl','wb') as f:
  pickle.dump(new_feat,f)
x_new = x[new_feat]

In [0]:
import pickle
new_feat = pickle.load(open('imp_feat.pkl','rb'))
x_new = x_train_scale[new_feat]
x_new_te = x_test_scale[new_feat]

In [0]:
sgd = SGDClassifier(loss='log',penalty='l2',learning_rate='adaptive',fit_intercept=False,alpha=0.0001,eta0=0.1,verbose=0)

In [0]:
sgd.fit(x_new,y_train)

y_pred =sgd.predict_proba(x_new_te)[:,1]
score = roc_auc_score(y_test,y_pred)

print('Auroc score: ',score)

Auroc score:  0.7624274382336401


**As we can see after removal of the zero importance features there is no significant improvement in score.**

## LGBMClassifier
Lightgbm is faster than XGBoost so we are using Lightgbm. Here Bayesian Optimization is used for hyperparameter tunning.

https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a
https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
https://datascience.stackexchange.com/questions/49567/lightgbm-vs-xgboost-vs-catboost

In [10]:
#https://github.com/hyperopt/hyperopt/issues/253#issuecomment-298960310

import lightgbm as lgb
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import tpe
from hyperopt import hp
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample

N_FOLDS = 10 
train_set = lgb.Dataset(x_train_scale, y_train)
def objective(params, n_folds = N_FOLDS):
  """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
  print('Param: ',params)
  # Perform n_fold cross validation with hyperparameters
  # Use early stopping and evalute based on ROC AUC
  cv_results = lgb.cv(params, train_set, nfold = n_folds, num_boost_round = 30, 
                      early_stopping_rounds = 100, metrics = 'auc', seed = 50,verbose_eval=True)

  # Extract the best score
  best_score = max(cv_results['auc-mean'])
  print('params: ',params)
  print('best score: ',best_score)
  # Loss must be minimized
  loss = 1 - best_score
  
  # Dictionary with information for evaluation
  return {'loss': loss, 'params': params, 'status': STATUS_OK}

space = {
    'num_leaves': sample(scope.int(hp.quniform('num_leaves', 30, 40, 1))),
    'max_depth':sample(scope.int(hp.quniform('max_depth', 5,10,1))),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.015), np.log(0.025)),
    'subsample': hp.uniform('subsample', 0.7,1.0),
    'min_child_weight': hp.uniform('min_child_weight', 20, 50),
    'reg_alpha': hp.uniform('reg_alpha', 0.01, 0.1),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 0.1),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'boosting_type':['goss']
}


# Algorithm
tpe_algorithm = tpe.suggest

# Trials object to track progress
bayes_trials = Trials()

from hyperopt import fmin

MAX_EVALS = 15

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)

Param: 
{'boosting_type': ('goss',), 'colsample_bytree': 0.647565964343581, 'learning_rate': 0.020375779094273946, 'max_depth': 7, 'min_child_weight': 28.672779515169474, 'num_leaves': 39, 'reg_alpha': 0.06335234940891847, 'reg_lambda': 0.08190157233934035, 'subsample': 0.7289578220377172}
[1]	cv_agg's auc: 0.682049 + 0.00844323
[2]	cv_agg's auc: 0.71836 + 0.007649
[3]	cv_agg's auc: 0.72122 + 0.00748529
[4]	cv_agg's auc: 0.72999 + 0.00673277
[5]	cv_agg's auc: 0.729965 + 0.00671997
[6]	cv_agg's auc: 0.731646 + 0.00643264
[7]	cv_agg's auc: 0.731163 + 0.00658066
[8]	cv_agg's auc: 0.735555 + 0.00658504
[9]	cv_agg's auc: 0.735277 + 0.00657114
[10]	cv_agg's auc: 0.735237 + 0.00631907
[11]	cv_agg's auc: 0.734882 + 0.00616827
[12]	cv_agg's auc: 0.735137 + 0.00613157
[13]	cv_agg's auc: 0.735151 + 0.00617212
[14]	cv_agg's auc: 0.735458 + 0.00609171
[15]	cv_agg's auc: 0.73529 + 0.00602044
[16]	cv_agg's auc: 0.735772 + 0.00591022
[17]	cv_agg's auc: 0.735658 + 0.00578119
[18]	cv_agg's auc: 0.737528

### Model training 
Model is trained using 10 fold cross validation. Parameter used here are found using above hyperparameter tunning.

'colsample_bytree': 0.6293241238277231, 'learning_rate': 0.015126331346361143, 'max_depth': 8, 'min_child_weight': 43.42940428346687, 'num_leaves': 40, 'reg_alpha': 0.0026698148330698726, 'reg_lambda': 0.007143177072986173, 'subsample': 0.9578756727736679

In [0]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold

folds = KFold(n_splits= 10, shuffle=True, random_state=1001)

preds = np.zeros(x_train_scale.shape[0])
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(x_train_scale,y_train)):
  train_x,train_y = x_train_scale.iloc[train_idx],y_train.iloc[train_idx]
  valid_x,valid_y = x_train_scale.iloc[valid_idx],y_train.iloc[valid_idx]
  clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.019258071793389385,
            num_leaves=33,
            colsample_bytree= 0.94060164457618193,
            subsample=0.87093236043578643,
            max_depth=9,
            reg_alpha= 0.041211518409727105,
            reg_lambda=0.07291283536418594,
            min_split_gain=0.0222415,
            min_child_weight=40.63404102198295,
            silent=-1,
            verbose=-1, )
  clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
  preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
  print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, preds[valid_idx])))
  del clf, train_x, train_y, valid_x, valid_y
  gc.collect()
print('Full AUC score %.6f' % roc_auc_score(y_train, preds))

Training until validation scores don't improve for 200 rounds.
[200]	training's auc: 0.796741	training's binary_logloss: 0.236172	valid_1's auc: 0.778329	valid_1's binary_logloss: 0.240501
[400]	training's auc: 0.82063	training's binary_logloss: 0.225918	valid_1's auc: 0.787557	valid_1's binary_logloss: 0.236373
[600]	training's auc: 0.837182	training's binary_logloss: 0.219112	valid_1's auc: 0.790585	valid_1's binary_logloss: 0.235143
[800]	training's auc: 0.850286	training's binary_logloss: 0.213679	valid_1's auc: 0.79144	valid_1's binary_logloss: 0.234745
[1000]	training's auc: 0.861851	training's binary_logloss: 0.208771	valid_1's auc: 0.792297	valid_1's binary_logloss: 0.234451
[1200]	training's auc: 0.872003	training's binary_logloss: 0.204206	valid_1's auc: 0.792843	valid_1's binary_logloss: 0.23427
[1400]	training's auc: 0.881243	training's binary_logloss: 0.19994	valid_1's auc: 0.793267	valid_1's binary_logloss: 0.234152
Early stopping, best iteration is:
[1344]	training's auc

Here highest score is in fold 4 with 0.7917 score.

Overall score is 0.7890

In [0]:
import pickle
new_feat = pickle.load(open('imp_feat.pkl','rb'))
x_new = x_train_scale[new_feat]
x_new_te = x_test_scale[new_feat]

In [0]:
x_new.head()

,NAME_INCOME_TYPE_Student,pos_NAME_CONTRACT_STATUS_Canceled_mean,buro_CREDIT_ACTIVE_Bad debt_mean,ORGANIZATION_TYPE_Industry: type 8,prev_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean,FLAG_DOCUMENT_2,ORGANIZATION_TYPE_Industry: type 10,prev_NAME_CASH_LOAN_PURPOSE_Money for a third person_mean,buro_CREDIT_TYPE_Loan for the purchase of equipment_mean,NAME_EDUCATION_TYPE_Academic degree,NAME_INCOME_TYPE_Maternity leave,pos_NAME_CONTRACT_STATUS_Amortized debt_mean,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Industry: type 6,FLAG_DOCUMENT_17,ORGANIZATION_TYPE_Transport: type 1,prev_NAME_GOODS_CATEGORY_Education_mean,prev_CHANNEL_TYPE_Car dealer_mean,ORGANIZATION_TYPE_Trade: type 5,prev_NAME_GOODS_CATEGORY_Weapon_mean,prev_NAME_PORTFOLIO_Cars_mean,FLAG_DOCUMENT_7,closed_CREDIT_DAY_OVERDUE_max,NAME_INCOME_TYPE_Unemployed,FLAG_DOCUMENT_15,prev_NAME_CASH_LOAN_PURPOSE_Buying a garage_mean,FLAG_DOCUMENT_14,prev_NAME_SELLER_INDUSTRY_Tourism_mean,FLAG_DOCUMENT_19,FLAG_DOCUMENT_13,prev_NAME_GOODS_CATEGORY_Fitness_mean,closed_CREDIT_DAY_OVERDUE_mean,prev_NAME_CASH_LOAN_PURPOSE_Hobby_mean,prev_NAME_GOODS_CATEGORY_Additional Service_mean,ORGANIZATION_TYPE_Industry: type 13,FLAG_DOCUMENT_20,prev_NAME_GOODS_CATEGORY_Insurance_mean,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Trade: type 1,...,ins_payment_perc_mean,buro_DAYS_CREDIT_min,ins_AMT_INSTALMENT_sum,ins_AMT_INSTALMENT_mean,ins_AMT_PAYMENT_mean,buro_DAYS_CREDIT_ENDDATE_mean,pos_MONTHS_BALANCE_mean,AMT_CREDIT,ins_payment_perc_sum,ins_AMT_PAYMENT_sum,ins_before_due_payment_max,approved_DAYS_DECISION_max,ins_AMT_PAYMENT_min,prev_DAYS_DECISION_max,active_DAYS_CREDIT_max,income_per_person,buro_DAYS_CREDIT_UPDATE_mean,approved_DAYS_DECISION_mean,prev_DAYS_DECISION_min,prev_DAYS_DECISION_mean,approved_DAYS_DECISION_min,ins_before_due_payment_sum,ins_AMT_PAYMENT_max,AMT_ANNUITY,ins_AMT_INSTALMENT_max,DAYS_LAST_PHONE_CHANGE,income_credit_per,ins_before_due_payment_mean,payment_rate,buro_DAYS_CREDIT_mean,DAYS_REGISTRATION,buro_DAYS_CREDIT_max,annuity_income_per,DAYS_ID_PUBLISH,emp_per,DAYS_EMPLOYED,DAYS_BIRTH,EXT_SOURCE_1,EXT_SOURCE_3,EXT_SOURCE_2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.999968,0.072895,0.013942,0.004049,0.003529,0.546517,0.473005,0.134775,0.999586,0.011076,0.029149,0.652859,0.000042,0.761041,0.912047,0.003967,0.953306,0.391895,0.142466,0.429892,0.142466,0.027353,0.007136,0.241897,0.007136,0.565780,0.001238,0.030597,0.714116,0.132900,0.599060,0.209446,0.188256,0.391969,0.344298,0.775789,0.531060,0.518870,0.987189,0.799724
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.999983,0.607803,0.001009,0.001172,0.001172,0.576205,0.757895,0.039326,0.999945,0.000920,0.020226,0.998973,0.000943,0.998973,0.912047,0.000793,0.990642,0.853425,0.707534,0.853425,0.707534,0.014436,0.002064,0.038094,0.002064,0.794872,0.001546,0.064594,0.272843,0.758214,0.608463,0.908624,0.079849,0.746005,0.352773,0.813519,0.698816,0.518870,0.696620,0.037746
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.999555,0.926762,0.017579,0.010891,0.010891,0.595090,0.874342,0.101124,0.997272,0.016022,0.007733,0.938377,0.005005,0.938377,0.926762,0.003159,0.999260,0.872945,0.773630,0.900837,0.773630,0.005224,0.010276,0.114502,0.010276,0.841121,0.002636,0.012466,0.374742,0.926762,0.754418,0.926762,0.057410,0.959844,0.194869,0.865553,0.478072,0.518870,0.598522,0.701248
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.999639,0.095483,0.047689,0.006924,0.006731,0.558565,0.671207,0.060892,0.990558,0.042253,0.018441,0.944

In [0]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold

folds = KFold(n_splits= 10, shuffle=True, random_state=1001)

preds = np.zeros(x_new.shape[0])
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(x_new,y_train)):
  train_x,train_y = x_new.iloc[train_idx],y_train.iloc[train_idx]
  valid_x,valid_y = x_new.iloc[valid_idx],y_train.iloc[valid_idx]
  clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.019258071793389385,
            num_leaves=33,
            colsample_bytree= 0.94060164457618193,
            subsample=0.87093236043578643,
            max_depth=9,
            reg_alpha= 0.041211518409727105,
            reg_lambda=0.07291283536418594,
            min_split_gain=0.0222415,
            min_child_weight=40.63404102198295,
            silent=-1,
            verbose=-1, )
  clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
  preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
  print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, preds[valid_idx])))
  del clf, train_x, train_y, valid_x, valid_y
  gc.collect()
print('Full AUC score %.6f' % roc_auc_score(y_train, preds))

Training until validation scores don't improve for 200 rounds.
[200]	training's auc: 0.796872	training's binary_logloss: 0.23617	valid_1's auc: 0.778545	valid_1's binary_logloss: 0.240479
[400]	training's auc: 0.820867	training's binary_logloss: 0.2259	valid_1's auc: 0.787584	valid_1's binary_logloss: 0.236387
[600]	training's auc: 0.837084	training's binary_logloss: 0.219169	valid_1's auc: 0.790804	valid_1's binary_logloss: 0.235144
[800]	training's auc: 0.850143	training's binary_logloss: 0.213804	valid_1's auc: 0.791818	valid_1's binary_logloss: 0.234655
[1000]	training's auc: 0.861641	training's binary_logloss: 0.20891	valid_1's auc: 0.792927	valid_1's binary_logloss: 0.234285
[1200]	training's auc: 0.872143	training's binary_logloss: 0.204231	valid_1's auc: 0.793606	valid_1's binary_logloss: 0.234038
[1400]	training's auc: 0.881486	training's binary_logloss: 0.199922	valid_1's auc: 0.79433	valid_1's binary_logloss: 0.233752
[1600]	training's auc: 0.889913	training's binary_logloss

In [0]:
lgb = LGBMClassifier()
lgb.fit(x_train_scale,y_train)
lgb.booster_.feature_importance()

fea_impotance = pd.DataFrame({'cols':x_train_scale.columns, 'fea_imp':lgb.feature_importances_})
fea_impotance.loc[fea_impotance.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

,cols,fea_imp
36,EXT_SOURCE_1,114
253,payment_rate,105
37,EXT_SOURCE_2,104
38,EXT_SOURCE_3,94
22,DAYS_BIRTH,66
...,...,...
390,prev_AMT_DOWN_PAYMENT_min,1
564,prev_PRODUCT_COMBINATION_POS other with intere...,1
561,prev_PRODUCT_COMBINATION_POS industry without ...,1
152,"NAME_TYPE_SUITE_Spouse, partner",1


In [0]:
new_train = x_train_scale[fea_impotance.loc[fea_impotance.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)['cols'].tolist()]

In [0]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold

folds = KFold(n_splits= 10, shuffle=True, random_state=1001)

preds = np.zeros(new_train.shape[0])
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(new_train,y_train)):
  train_x,train_y = new_train.iloc[train_idx],y_train.iloc[train_idx]
  valid_x,valid_y = new_train.iloc[valid_idx],y_train.iloc[valid_idx]
  clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.019258071793389385,
            num_leaves=33,
            colsample_bytree= 0.94060164457618193,
            subsample=0.87093236043578643,
            max_depth=9,
            reg_alpha= 0.041211518409727105,
            reg_lambda=0.07291283536418594,
            min_split_gain=0.0222415,
            min_child_weight=40.63404102198295,
            silent=-1,
            verbose=-1, )
  clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
  preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
  print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, preds[valid_idx])))
  del clf, train_x, train_y, valid_x, valid_y
  gc.collect()
print('Full AUC score %.6f' % roc_auc_score(y_train, preds))

Training until validation scores don't improve for 200 rounds.
[200]	training's binary_logloss: 0.236124	training's auc: 0.796989	valid_1's binary_logloss: 0.240573	valid_1's auc: 0.777882
[400]	training's binary_logloss: 0.225885	training's auc: 0.820843	valid_1's binary_logloss: 0.23633	valid_1's auc: 0.787678
[600]	training's binary_logloss: 0.219126	training's auc: 0.837204	valid_1's binary_logloss: 0.235156	valid_1's auc: 0.790571
[800]	training's binary_logloss: 0.213673	training's auc: 0.850309	valid_1's binary_logloss: 0.234745	valid_1's auc: 0.791588
[1000]	training's binary_logloss: 0.208687	training's auc: 0.861896	valid_1's binary_logloss: 0.234366	valid_1's auc: 0.792646
[1200]	training's binary_logloss: 0.204021	training's auc: 0.872406	valid_1's binary_logloss: 0.234166	valid_1's auc: 0.793146
[1400]	training's binary_logloss: 0.199538	training's auc: 0.882006	valid_1's binary_logloss: 0.234192	valid_1's auc: 0.792988
Early stopping, best iteration is:
[1273]	training's 

In [0]:
test = test[x_train_scale.columns.tolist()]
test.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,...,cc_AMT_DRAWINGS_CURRENT_min,cc_AMT_DRAWINGS_CURRENT_max,cc_AMT_DRAWINGS_CURRENT_mean,cc_AMT_DRAWINGS_CURRENT_sum,cc_AMT_DRAWINGS_OTHER_CURRENT_sum,cc_AMT_DRAWINGS_POS_CURRENT_sum,cc_AMT_INST_MIN_REGULARITY_min,cc_AMT_INST_MIN_REGULARITY_max,cc_AMT_INST_MIN_REGULARITY_mean,cc_AMT_INST_MIN_REGULARITY_sum,cc_AMT_PAYMENT_CURRENT_sum,cc_AMT_PAYMENT_TOTAL_CURRENT_min,cc_AMT_PAYMENT_TOTAL_CURRENT_max,cc_AMT_PAYMENT_TOTAL_CURRENT_mean,cc_AMT_PAYMENT_TOTAL_CURRENT_sum,cc_AMT_RECEIVABLE_PRINCIPAL_min,cc_AMT_RECEIVABLE_PRINCIPAL_max,cc_AMT_RECEIVABLE_PRINCIPAL_mean,cc_AMT_RECEIVABLE_PRINCIPAL_sum,cc_AMT_RECIVABLE_min,cc_AMT_RECIVABLE_max,cc_AMT_RECIVABLE_mean,cc_AMT_RECIVABLE_sum,cc_AMT_TOTAL_RECEIVABLE_min,cc_AMT_TOTAL_RECEIVABLE_max,cc_AMT_TOTAL_RECEIVABLE_mean,cc_AMT_TOTAL_RECEIVABLE_sum,cc_CNT_DRAWINGS_ATM_CURRENT_sum,cc_CNT_DRAWINGS_CURRENT_sum,cc_CNT_DRAWINGS_OTHER_CURRENT_sum,cc_CNT_DRAWINGS_POS_CURRENT_sum,cc_CNT_INSTALMENT_MATURE_CUM_sum,cc_SK_DPD_min,cc_SK_DPD_max,cc_SK_DPD_mean,cc_SK_DPD_sum,cc_SK_DPD_DEF_min,cc_SK_DPD_DEF_max,cc_SK_DPD_DEF_mean,cc_SK_DPD_DEF_sum
307507,20560.5,568800.0,450000.0,135000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0660,0.0666,0.0672,0.0590,0.0590,0.0612,0,2.0,1,NaN,NaN,NaN,-19241,-2329.0,-812,-1740.0,-5170.0,0.0,0.0,NaN,NaN,NaN,0,0.1379,0.1379,0.1379,0.752614,0.789654,0.159520,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,17370.0,222768.0,180000.0,99000.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,0,NaN,NaN,NaN,-18064,-4469.0,-1623,0.0,-9118.0,0.0,0.0,NaN,NaN,NaN,-1,NaN,NaN,NaN,0.564990,0.291656,0.432962,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307509,69777.0,663264.0,630000.0,202500.0,0.0,0.0,0.0,1.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,0,NaN,NaN,NaN,-20038,-4458.0,-3503,-856.0,-2175.0,0.0,0.0,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,0.699787,0.610991,1,...,0.0,157500.00,5953.125000,571500.00,0.0,0.00,0.0,7875.0,1454.539551,129454.020,688161.24,0.0,153675.0,6817.172344,654448.545,0.0,157500.000,17255.559844,1656533.745,-274.32,161420.220,18101.079844,1737703.665,-274.32,161420.220,18101.079844,1737703.665,23.0,23.0,0.0,0.0,1666.0,0.0,1.0,0.010417,1.0,0.0,1.0,0.010417,1.0
307510,49018.5,1575000.0,1575000.0,315000.0,0.0,0.0,0.0,0.0,0.0,3.0,0.3052,0.3081,0.3109,0.1974,0.1974,0.2049,2,4.0,1,0.1165,0.1173,0.1176,-13976,-1866.0,-4208,-1805.0,-2000.0,0.0,0.0,0.32,0.32,0.3222,0,0.2759,0.2759,0.2759,0.525734,0.509677,0.612704,1,...,0.0,22823.55,6156.400408,301663.62,0.0,274663.62,0.0,11250.0,6133.363929,257601.285,283724.46,0.0,15750.0,5606.152347,274701.465,0.0,36270.675,7680.352041,376337.250,0.00,36980.415,7968.609184,390461.850,0.00,36980.415,7968.609184,390461.850,2.0,117.0,0.0,115.0,821.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
307511,32067.0,625500.0,625500.0,180000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0,NaN,NaN,NaN,-13040,-2191.0,-4262,-821.0,-4000.0,0.0,0.0,NaN,NaN,NaN,-1,NaN,NaN,NaN,0.202145,0.425687,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Conclusion

In [0]:
from prettytable import PrettyTable
pt=PrettyTable()
pt.field_names=["model","test_auc"]
pt.add_row(["Logistic Regression","0.7640"])
pt.add_row(["SGD with log loss","0.7624"])
pt.add_row(["SGD after feature removal","0.7624"])
pt.add_row(["LGBMClassifier","0.7890"])
print(pt)

+---------------------------+----------+
|           model           | test_auc |
+---------------------------+----------+
|    Logistic Regression    |  0.7640  |
|     SGD with log loss     |  0.7624  |
| SGD after feature removal |  0.7624  |
|       LGBMClassifier      |  0.7890  |
+---------------------------+----------+


Here LGBM Classifier performes well with highest accuracy